In [1]:
import pandas as pd

In [31]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
ids = test['id']
df = pd.concat([train, test])

In [32]:
df = df.drop(columns = ['id'])

In [33]:
for col in df.columns:
    df[col] = df[col].fillna(df[col].mode()[0])

In [34]:
df['bin_3'] = df['bin_3'].map({'F': 0, 'T': 1})
df['bin_4'] = df['bin_4'].map({'N': 0, 'Y': 1})

In [35]:
ord1dict = {'Novice': 0, 'Contributor': 1, 'Expert': 2, 'Master': 3, 'Grandmaster': 4}
ord2dict = {'Freezing': 0, 'Cold': 1, 'Warm': 2, 'Hot': 3, 'Boiling Hot': 4, 'Lava Hot': 5}

df['ord_1'] = df['ord_1'].map(ord1dict)
df['ord_2'] = df['ord_2'].map(ord2dict)

In [36]:
from sklearn.preprocessing import OrdinalEncoder

ord_345 = ['ord_' + str(i) for i in range(3, 6)]

ord_encoder = OrdinalEncoder()

df[ord_345] = ord_encoder.fit_transform(df[ord_345])


In [37]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()
nom_features = ['nom_' + str(i) for i in range(10)]
nom_dummy = encoder.fit_transform(df[nom_features])

df_non_nom = df.drop(columns = nom_features)

In [38]:
day_dict = {i: np.sin(k) for i, k in enumerate(np.linspace(0, np.pi / 2, df_non_nom['day'].nunique() + 1))}
month_dict = {i: np.sin(k) for i, k in enumerate(np.linspace(0, np.pi / 2, df_non_nom['month'].nunique() + 1))}

df_non_nom['day'] = df_non_nom['day'].map(day_dict)
df_non_nom['month'] = df_non_nom['month'].map(month_dict)

In [39]:
from sklearn.preprocessing import MinMaxScaler
ord_features = ['ord_' + str(i) for i in range(6)]
df_non_nom[ord_features] = MinMaxScaler().fit_transform(df_non_nom[ord_features])

In [51]:
from scipy import sparse
X = sparse.hstack([sparse.csr_matrix(df_non_nom.drop(columns = 'target')),
                               encoded_nom_matrix,
                              ],
                              format='csr')

In [52]:
X_train = X[:len(train)]
X_test = X[len(train):]
y = train['target']

In [53]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y, test_size=0.3)

In [59]:
from sklearn.linear_model import LogisticRegression

logistic_model = LogisticRegression(C = 0.1, max_iter = 1000)
logistic_model.fit(X_train, y_train)

LogisticRegression(C=0.1, max_iter=1000)

In [60]:
y_valid_preds = logistic_model.predict(X_valid)

In [61]:
from sklearn.metrics import roc_auc_score

In [62]:
print(roc_auc_score(y_valid, y_valid_preds))

0.5828101606417095
